# Check Calculation Results

# 1. Preparation

## 1.1. Import Standard Library

In [ ]:
import pandas as pd
# Progress bar
from tqdm import tqdm_notebook
# Datetime format
from datetime import datetime
# Calculate time difference between given dates.
from dateutil.relativedelta import relativedelta

## 1.2. Import SQL Library

In [ ]:
import sqlite3
# Go through given folder and return the list of files.
import glob
import csv

## 1.3. Import Excel Library

In [ ]:
# Automate Excel with Python
import xlwings as xw

# 2. Check Original Data
- Check if the original data is imported into Excel correctly.

## 2.1. Create Database

In [ ]:
# Define Function to Execute Database Commands.
def execute_db(fname, sql_cmd):
    # Establish a connection to SQLite database by creating a connection object "conn."
    conn = sqlite3.connect(fname)
    # Create a cursor object using the cursor method of "conn."
    c = conn.cursor()
    # Call the cursor’s execute method. 
    c.execute(sql_cmd)
    # Save the changes.
    conn.commit()
    # Close the connection.
    conn.close()

In [ ]:
# Define Function to Select Data from Database.
def select_db(fname, sql_cmd):
    # Establish a connection to SQLite database by creating a connection object "conn."
    conn = sqlite3.connect(fname)
    # Create a cursor object using the cursor method of "conn."
    c = conn.cursor()
    # Call the cursor’s execute method. 
    c.execute(sql_cmd)
    # Fetch all rows of a query result.
    rows = c.fetchall()
    # Close the connection.
    conn.close()
    # Return query results. 
    return rows

In [ ]:
# Create a Blank Database
# Create a SQLite database file named "db." 
db_name = "db.sqlite"
# Create a table named "Data."
cmd = ("CREATE TABLE Data "
       "(id INTEGER PRIMARY KEY AUTOINCREMENT, "
       "Time DATETIME, Category TEXT, Channel TEXT, Sales REAL)")
# Execute function "execute_db" with "db_name" and "sql_cmd" being specified. 
execute_db(db_name, cmd)

In [ ]:
# Import Data into Database
# Find all files in the specific directory. 
files = glob.glob("G:\\Data1\\*.xls")

for file in files:
    # Import data from all the xls files. 
    data = pd.read_excel(file)
    # Establish a connection to SQLite database "db."
    con = sqlite3.connect("db.sqlite")
    # Write records stored in dataframe "data" to table "Data" in SQLite database "db."
    # If table "Data" already exists, insert new values to the existing table.
    data.to_sql("Data", con, index = False, if_exists = "append")
    # Save the changes.
    con.commit()
    # Close the connection.
    con.close() 

## 2.2. Import Database Data

In [ ]:
# Establish a connection to SQLite database "db."
conn_data = sqlite3.connect("db.sqlite")
# Create a cursor object using the cursor method of "conn."
cursor_data = conn_data.cursor()
# Save the changes.
conn_data.commit()
# Return all records from table "Data" in "db." 
data = pd.read_sql_query('''
                         SELECT *
                         FROM main.Data
                         ''', conn_data)
# Close the connection.
conn_data.close()

In [ ]:
# Convert strings to datetime.
data["Time"] = pd.to_datetime(data["Time"])
Check data type.
data.dtypes

## 2.3. Import Excel Data

In [ ]:
# Run excel in the background.
xw.App().visible = False

In [ ]:
# Detemine which workbook to activate.
wb = xw.Book("performance.xlsm")

In [ ]:
# Activate a sheet based on sheet name.
sheet1 = wb.sheets["Sheet1"]
sheet2 = wb.sheets["Sheet2"]
sheet3 = wb.sheets["Sheet3"]
sheet4 = wb.sheets["Sheet4"]

## 2.4. Compare Calculation Results

### 2.4.1. Set Parameters

In [ ]:
# "a_list" is a list of lists which contain "major category", "intermediary category" and "subcategory" sequentially.
a_list = sheet1.range("B4:D99").value

In [ ]:
# Specify which month to be applied with calculation checking.
today = "2019/10/1"
last_y = "2018/10/1"
last_m = "2019/9/1"

# Get data of the given month from database.
d_td = datetime.strptime(today, "%Y/%m/%d")
d_ly = datetime.strptime(last_y, "%Y/%m/%d")
d_lm = datetime.strptime(last_m, "%Y/%m/%d")

# Get data of the given month from Excel.
sheet2.range("E5").value = today
sheet2.range("E6").value = last_y
sheet2.range("E7").value = last_m

# Define channels and subchannels.
channel_b = ["Offline", "Online"]
channel_s = ["Offline_A", "Offline_B", "Offline_Total", "Online_C", "Online_D"]

### 2.4.2. Check Monthly Sales Number

In [ ]:
# Foe each category:
for a in tqdm_notebook(a_list):
    # "a_list" is a list of lists which contains "major category", "intermediary category" and "subcategory" sequentially.    
    sheet2.range("H5").value = a[0]
    sheet2.range("H6").value = a[1]
    sheet2.range("H7").value = a[2]
    
    # For each channel:
    for cb in channel_b:
        for cs in channel_s:      
            sheet2.range("K6").value = cb
            sheet2.range("K7").value = cs
        
            # Get monthly sales data from Excel.
            excel_td = sheet2.range("E10").value
            excel_ly = sheet2.range("E11").value
            excel_lm = sheet2.range("E12").value           

            # Get monthly sales data from database.
            sql_td_v = data[(data["Time"] == d_td) & (data["Category"] == a[2]) & (data["Channel"] == cs)]["Sales"]
            sql_ly_v = data[(data["Time"] == d_ly) & (data["Category"] == a[2]) & (data["Channel"] == cs)]["Sales"]
            sql_lm_v = data[(data["Time"] == d_lm) & (data["Category"] == a[2]) & (data["Channel"] == cs)]["Sales"]
            
            if sql_td_v.empty or sql_ly_v.empty or sql_lm_v.empty:
                continue
            else:                
                sql_td = float(sql_td_v)
                sql_ly = float(sql_ly_v)
                sql_lm = float(sql_lm_v)
            
            # Compare calculation results.
            if (excel_td != sql_td) or (excel_ly != sql_ly) or (excel_lm != sql_lm):
                print("Error!")
                print("Category:", a[2])
                print("Channel:", cb)
                print("Sub-Channel:", cs) 
                print("Excel: Sales of This Month:", excel_td) 
                print("Database: Sales of This Month:", sql_td)
                print("Excel: Sales of Last Year:", excel_ly) 
                print("Database: Sales of Last Year:", sql_ly)
                print("Excel: Sales of Previous Month:", excel_lm) 
                print("Database: Sales of Previous Month:", sql_lm) 
                print("=================")

### 2.4.3.  Check Cumulative Sales Number

In [ ]:
# Foe each category:
for a in tqdm_notebook(a_list):    
    
    sheet2.range("H5").value = a[0]
    sheet2.range("H6").value = a[1]
    sheet2.range("H7").value = a[2]
    
    # For each channel:
    for cb in channel_b:
        for cs in channel_s:      
            sheet2.range("K6").value = cb
            sheet2.range("K7").value = cs
        
            # Get cumulative sales data from Excel.
            excel_ac_td = int(round(sheet2.range("E35").value, 0))
            excel_ac_ly = int(round(sheet2.range("E51").value, 0))        

            # Get cumulative sales data from SQLite database.
            sql_ac_td = []
            # i = 0,1,2,3,4,5,6,7,8,9
            for i in range(d_td.month):
                # date = 10-0=Oct, 10-1=Sep, 10-2=Aug, 10-3=Jul, 10-4=Jun, 10-5=May, 10-6=Apr, 10-7=Mar, 10-8=Feb, 10-9=Jan
                date = d_td - relativedelta(months=i)
                v = data[(data["Time"] == date) & (data["Category"] == a[2]) & (data["Channel"] == cs)]["Sales"]                
                if v.empty:
                    sql_ac_td.append(v)
                else:
                    sql_ac_td.append(float(v))
                  
            sql_ac_ly = []
            for i in range(d_ly.month):
                date = d_ly - relativedelta(months=i)
                v = data[(data["Time"] == date) & (data["Category"] == a[2]) & (data["Channel"] == cs)]["Sales"]
                if v.empty:
                    sql_ac_ly.append(v)
                else:
                    sql_ac_ly.append(float(v))                             
            
            # Compare calculation results.            
            # When there is an empty series in a list, summing elements of a list will return a series.
            # Therefore, by checking if the datatpye of sum of elements in a list is series or not, 
            # we can confirm whether there is correspoing values of given properties.
            if (type(sum(sql_ac_td)) == type(pd.Series([]))) or (type(sum(sql_ac_ly)) == type(pd.Series([]))):
                continue
            
            if (excel_ac_td != int(round(sum(sql_ac_td), 0))) or (excel_ac_ly != int(round(sum(sql_ac_ly), 0))): 
                print("Error!")
                print("Category:", a[2])
                print("Channel:", cb)
                print("Sub-Channel:", cs) 
                print("Excel: Cumulative Sales of This Year:", excel_ac_td) 
                print("Excel: Cumulative Sales of Last Year:", excel_ac_ly) 
                print("Database: Cumulative Sales of This Year:", int(round(sum(sql_ac_td), 0)))
                print("Database: Cumulative Sales of Last Year:", int(round(sum(sql_ac_ly), 0)))              
                print("=================")

# 3. Check Calculation
- Check if the calculation run by VBA is correct by making the comparison with the results calculated by Excel formula.

In [ ]:
# Excel Sheet1: Calculation run by VBA
# For each category:
for a in tqdm_notebook(a_list):
    # Find the location of each category and its corresponding monthly sales number.    
    # Since "api.UsedRange.Find" only returns the location of the first string containing a specific substring,
    # for those substrings shared by multiple categories, additional string matching has to be applied.
    # Example: "Headset" and "Bluetooth Headset", "Toothbrush" and "Electric Toothbrush"     
    if a[2] in ['Headset', 'Toothbrush']:
        for index, value in enumerate(sheet3.range('B1:B120').value):
            if value == a[2]:
                offline = 'F' + str(index+1)
                online = 'J' + str(index+1)
                online_c = 'N' + str(index+1)
                online_d = 'T' + str(index+1)
    
    else:
        r = sheet3.api.UsedRange.Find(a[2])
        if r == None:
            continue
        else:
            s = sheet3.api.UsedRange.Find(a[2]).address
            offline = 'F' + s.split('$B$')[1]
            online = 'J' + s.split('$B$')[1]
            online_c = 'N' + s.split('$B$')[1]
            online_d = 'T' + s.split('$B$')[1]

    vba_offline = round(sheet3.range(offline).value, 2)
    vba_online = round(sheet3.range(online).value, 2)
    vba_online_c = round(sheet3.range(online_c).value, 2)
    vba_online_d = round(sheet3.range(online_d).value, 2)
        
    # Excel Sheet2: Calculation run by Formula
    # Set category
    sheet2.range('H5').value = a[0]
    sheet2.range('H6').value = a[1]
    sheet2.range('H7').value = a[2]   
    
    # Set channel
    sheet2.range('K6').value = 'Offline'
    sheet2.range('K7').value = 'Offline_Total'
    formula_offline = round(sheet2.range('E13').value, 2)
    
    # Check each subcategory's corresponding monthly sales numbers.
    sheet2.range('K6').value = 'Online'
    sheet2.range('K7').value = 'Online_Total'
    formula_online = round(sheet2.range('E13').value, 2)
    sheet2.range('K7').value = 'Online_C'
    formula_online_c = round(sheet2.range('E13').value, 2)
    sheet2.range('K7').value = 'Online_D'  
    formula_online_d = round(sheet2.range('E13').value, 2)
    
    # Compare calculation results.   
    if (vba_offline != formula_offline) or (vba_online != formula_online) or \
       (vba_online_c != formula_online_c) or (vba_online_d != formula_online_d):
            print(a[2]) 
            print(vba_offline, formula_offline)
            print(vba_online, formula_online)
            print(vba_online_c, formula_online_c)
            print(vba_online_d, formula_online_d)

# 4. Check Data Location
- Check if monthly sales related numbers of given category are shown at the desired cells.

In [ ]:
# Get sales numbers of corresponding category from Excel Sheet1.
def sheet1_get_sales(sheet, col, index):
    a = sheet.range(col + str(index + 21)).value
    b = sheet.range(col + str(index + 23)).value
    return a, b

In [ ]:
# Get market shares of corresponding category from Excel Sheet1.
def sheet1_get_share(sheet, col, index):
    a = sheet.range(col + str(index + 21)).value
    b = sheet.range(col + str(index + 22)).value
    c = sheet.range(col + str(index + 23)).value
    d = sheet.range(col + str(index + 24)).value
    return a, b, c, d

In [ ]:
# Get sales numbers and market shares of corresponding category from Excel Sheet2.
def sheet2_get_nums(sheet, item):
    sheet.range('K7').value = item
    a = sheet.range('H12').value
    b = sheet.range('K12').value
    c = sheet.range('H20').value
    d = sheet.range('K20').value
    e = sheet.range('H19').value
    f = sheet.range('K19').value
    g = sheet.range('E10').value
    h = sheet.range('E14').value
    i = sheet.range('E13').value  
    return a, b, c, d, e, f, g, h, i

In [ ]:
# Compare numbers derived from Excel Sheet1 and Sheet2 to verify calculated results are at the desired locations.
def check(data1, data2):
    if data1 != data2:
        print(data1, data2)  

In [ ]:
# Excel Sheet1: How the results are presented.
index = 37
sheet = sheet4

# Excel Sheet2: Set specific time, channel and category to get corresponding numbers.
# "b_list" is a list of lists which contain "major category", "intermediary category" and "subcategory" sequentially.
b_list_index = 0

# Excel Sheet1: 
# Set category
# "c_list" is a list of lists which contain "intermediary category" and "subcategory" sequentially.
for c in c_list:
    for i in tqdm_notebook(c):        
        sheet.range('T' + str(index)).value = i

        # Get market shares
        sheet1_online_c_online_share, sheet1_online_d_online_share, \
        sheet1_online_c_panel_share, sheet1_online_d_panel_share = get_share(sheet, 'V', index)

        # Get Month-over-Month growth rate of market shares
        sheet1_online_c_online_share_mom, sheet1_online_d_online_share_mom, \
        sheet1_online_c_panel_share_mom, sheet1_online_d_panel_share_mom = sheet1_get_share(sheet, 'X', index)        

        # Get Year-over-Year growth rate of market shares
        sheet1_online_c_online_share_yoy, sheet1_online_d_online_share_yoy, \
        sheet1_online_c_panel_share_yoy, sheet1_online_d_panel_share_yoy = sheet1_get_share(sheet, 'Z', index)

        # Get sales numbers
        sheet1_offline_c_sales, sheet1_offline_d_sales = sheet1_get_sales(sheet, 'AD', index)

        # Get Month-over-Month growth rate of sales
        sheet1_online_c_sales_mom, sheet1_online_d_sales_mom = sheet1_get_sales(sheet, 'AF', index)

        # Get Year-over-Year growth rate of sales
        sheet1_online_c_sales_yoy, sheet1_online_d_sales_yoy = sheet1_get_sales(sheet, 'AH', index)       

        # Excel Sheet2:
        # Set category
        sheet2.range('H5').value = b_list[b_list_index][0] 
        sheet2.range('H6').value = b_list[b_list_index][1]
        sheet2.range('H7').value = b_list[b_list_index][2]
        
        # Get sales numbers and market shares of corresponding category sold via offline_c.
        sheet2_online_c_online_share, sheet2_online_c_panel_share, \
        sheet2_online_c_online_share_mom, sheet2_online_c_pannel_share_mom, \
        sheet2_online_c_online_share_yoy, sheet2_online_c_pannel_share_yoy, \
        sheet2_online_c_sales, sheet2_online_c_sales_mom, sheet2_online_c_sales_yoy = sheet2_get_nums(sheet2, 'Offline_C')

        # Get sales numbers and market shares of corresponding category sold via offline_d.
        sheet2_online_d_online_share, sheet2_online_d_panel_share, \
        sheet2_online_d_online_share_mom, sheet2_online_d_pannel_share_mom, \
        sheet2_online_d_online_share_yoy, sheet2_online_d_pannel_share_yoy, \
        sheet2_online_d_sales, sheet2_online_d_sales_mom, sheet2_online_d_sales_yoy = sheet2_get_nums(sheet2, 'Offline_C')

        # Compare results derived from 2 sheets.
        check(round(sheet1_online_d_online_share, 4), round(sheet2_online_d_online_share, 4))
        check(round(sheet1_online_c_panel_share, 4), round(sheet2_online_c_panel_share, 4))
        check(round(sheet1_online_c_online_share_mom, 4), round(sheet2_online_c_online_share_mom, 4))
        check(round(sheet1_online_c_pannel_share_mom 4), round(sheet2_online_c_pannel_share_mom, 4))
        check(round(sheet1_online_c_online_share_yoy, 4), round(sheet2_online_c_online_share_yoy, 4))
        check(round(sheet1_online_c_pannel_share_yoy, 4), round(sheet2_online_c_pannel_share_yoy, 4))
        check(round(sheet1_online_c_sales, 4), round(sheet2_online_c_sales, 4))
        check(round(sheet1_online_c_sales_mom, 4), round(sheet2_online_c_sales_mom, 4))
        check(round(sheet1_online_c_sales_yoy, 4), round(sheet2_online_c_sales_yoy, 4))
        
        check(round(sheet1_online_d_online_share, 4), round(sheet2_online_d_online_share, 4))
        check(round(sheet1_online_d_panel_share, 4), round(sheet2_online_d_panel_share, 4))
        check(round(sheet1_online_d_online_share_mom, 4), round(sheet2_online_d_online_share_mom, 4))
        check(round(sheet1_online_d_pannel_share_mom, 4), round(sheet2_online_d_pannel_share_mom, 4))
        check(round(sheet1_online_d_online_share_yoy, 4), round(sheet2_online_d_online_share_yoy, 4))
        check(round(sheet1_online_d_pannel_share_yoy, 4), round(sheet2_online_d_pannel_share_yoy, 4))
        check(round(sheet1_online_d_sales, 4), round(sheet2_online_d_sales, 4))
        check(round(sheet1_online_d_sales_mom, 4), round(sheet2_online_d_sales_mom, 4))
        check(round(sheet1_online_d_sales_yoy, 4), round(sheet2_online_d_sales_yoy, 4))
    
        # Move to next "subcategory"
        b_list_index = b_list_index + 1
    # Move to next "intermediary category" 
    index = index + 28    